# CS 5010
## Learning Python (Python version: 3)
### Topics:
###   - More examples about Aggregates and Group Operations

### Filename: Module 10 - Aggregates (Part III - More Examples)

# Data Aggregation and Group Operations

#### From "Python for Data Analysis" textbook (Wes McKinney; O'Reilly)

In [ ]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
# import matplotlib.pyplot as plt
import matplotlib as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## GroupBy Mechanics

In [ ]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


#### select the data (e.g. column 'data1') and then select the grouping (followed by an aggregate function)

For example: grouped = df['data1'].groupby(df['key1'])

Here the data to be used comes from the 'data1' column, and the grouping is based on 'key1'. Note there are three a's and two b's. So there will be two groups.

Notice what is produced - a SeriesGroupBy object. Behind the scenes it has done the grouping as described. Now it awaits the operation; so we can implement the sum() or mean() aggregate function for the final result. 

In [ ]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [ ]:
# Sum aggregate
grouped.sum()

key1
a    2.240016
b   -1.075169
Name: data1, dtype: float64

In [ ]:
#mean aggregate
grouped.mean()

key1
a    0.746672
b   -0.537585
Name: data1, dtype: float64

In [ ]:
# All in one step
# This time, using 'data1' but grouping first by 'key1' THEN by 'key2'
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.880536
      two     0.478943
b     one    -0.519439
      two    -0.555730
Name: data1, dtype: float64

In [ ]:
# Restructuring the output, using unstack(): It means moving the innermost row index to become the innermost column index. 
means.unstack()

key2,one,two
key1,,
a,0.880536,0.478943
b,-0.519439,-0.555730


Read more about reshaping dataframe in Chapter 8 [McKinney] and [blog post](https://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/)

In [ ]:
# In this case, still using df values from 'data1' column, but now grouping by states and years as shown below
# Then applying mean() aggregate function
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.478943
            2006   -0.519439
Ohio        2005   -0.380219
            2006    1.965781
Name: data1, dtype: float64

In [ ]:
# will take the df, and group by 'key1', then apply mean
# So will show mean for each grouping of 'key1' for data1 and data2
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.746672,0.910916
b,-0.537585,0.525384


In [ ]:
# Previous query:  df.groupby('key1').mean()
# Now grouping by 'key1' then 'key2', then apply mean
# So will show mean for each grouping of 'key1' and 'key2' for data1 and data2
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.880536  1.319920
     two   0.478943  0.092908
b    one  -0.519439  0.281746
     two  -0.555730  0.769023

In [ ]:
# Applying size() here counts the number of instances of the grouping
# That is, how many times did you see key1=a AND key2=one? If you look at original df, you'll see two (2) occuranaces
# In contrast, there is only one (1) occurance of key1=b AND key2=one
# Note: any missing values in a group key will be excluded from the result

df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data

Consider the following. (First, only printing 'name' to better understand what is going on, then, the full statement)

In [ ]:
# This piece of code is to break down the next query so we can better understand it, 'name' here pertains to the values in 'key1'
for name, group in df.groupby('key1'):
    print(name)

a
b


In [ ]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [ ]:
# Grouping by 'key1' which has values 'a' and 'b'. These become 'name'. 
# The associated data that goes with them is represented in the chunk of data ('group')

for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435
b
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023


In the case of multiple keys, the first element in the tuple will be a tuple of key values

In [ ]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)
    print("---") # Added for clarity to see the chunking ('group') that is happening; data associated with (k1, k2)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
4    a  one  1.965781  1.246435
---
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.478943  0.092908
---
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
---
('b', 'two')
  key1 key2    data1     data2
3    b  two -0.55573  0.769023
---


The rest of the section in this chapter has some other interesting ways to group and collect data 
including computing a dictionary (dict) of the data pieces, and a few ways to print out the groups:

In [ ]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023


In [ ]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [ ]:
df.dtypes
grouped = df.groupby(df.dtypes, axis=1)

In [ ]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -0.204708  1.393406
1  0.478943  0.092908
2 -0.519439  0.281746
3 -0.555730  0.769023
4  1.965781  1.246435
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect f column subsettting for aggregation.

This means

`df.groupby('key1')['data1']`  is equivalent to  `df['data1'].groupby(df['key1'])`

and

`df.groupby('key1')[['data2']]`  is equivalent to  `df[['data2']].groupby(df['key1'])`

Especially for large datasets, it may be desirable to aggregate only a few columns.

For example, to compute means for just the 'data2' column and get the result as a DataFrame, you could write the following

In [ ]:
df['data2']

0    1.393406
1    0.092908
2    0.281746
3    0.769023
4    1.246435
Name: data2, dtype: float64

In [ ]:
# dict(list(df.groupby(['key1', 'key2'])[['data2']]))

In [ ]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   1.319920
     two   0.092908
b    one   0.281746
     two   0.769023

In [ ]:
# Group by 'key1' and 'key2' only on column 'data2'
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped
s_grouped.mean()

key1  key2
a     one     1.319920
      two     0.092908
b     one     0.281746
      two     0.769023
Name: data2, dtype: float64

### Grouping with Dicts and Series

In [ ]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,1.007189,-1.296221,0.274992,0.228913,1.352917
Steve,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Wes,-0.539741,NaN,NaN,-1.021228,-0.577087
Jim,0.124121,0.302614,0.523772,0.000940,1.343810
Travis,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [ ]:
# group correspondence for the columns
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}
# included 'f' to highlight that unused grouping keys are OK

In [ ]:
by_column = people.groupby(mapping, axis=1) # pass the dictionary
by_column.sum()

,blue,red
Joe,0.503905,1.063885
Steve,1.297183,-1.553778
Wes,-1.021228,-1.116829
Jim,0.524712,1.770545
Travis,-4.230992,-2.405455


In [ ]:
# can work for Series
map_series = pd.Series(mapping)
map_series
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

Using Python functions is more generic way of defining a group mapping. Any function passed as a group key will be called once per index value, with the return values being used as the group names.

Let's use the people's first names as index values (from the previous example -- Joe, Steve, ... etc)

Let's group by the length of the names - you can just pass the 'len' function as seen next

In [ ]:
people

,a,b,c,d,e
Joe,1.007189,-1.296221,0.274992,0.228913,1.352917
Steve,0.886429,-2.001637,-0.371843,1.669025,-0.438570
Wes,-0.539741,NaN,NaN,-1.021228,-0.577087
Jim,0.124121,0.302614,0.523772,0.000940,1.343810
Travis,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [ ]:
people.groupby(len).sum()

# you see that there are three name lengths (3, 5, and 6) and this is how the grouping has been created

,a,b,c,d,e
3,0.591569,-0.993608,0.798764,-0.791374,2.119639
5,0.886429,-2.001637,-0.371843,1.669025,-0.438570
6,-0.713544,-0.831154,-2.370232,-1.860761,-0.860757


In [ ]:
# Mixing functions with arrays, dicts, or Series is not a problem

key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

# You'll see still grouping by length of name (3, 5, and 6) but then also by key_list ('one' and 'two')

a         b         c         d         e
3 one -0.539741 -1.296221  0.274992 -1.021228 -0.577087
  two  0.124121  0.302614  0.523772  0.000940  1.343810
5 one  0.886429 -2.001637 -0.371843  1.669025 -0.438570
6 two -0.713544 -0.831154 -2.370232 -1.860761 -0.860757

## Data Aggregation

Note the "Optimized groupby methods" table in the text (section: Data Aggregation, 10.2)
Many common aggregations have optimized implementations, but you're not limited to only this set

In [ ]:
df # Reminder of data set

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [ ]:
df
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.668413
b   -0.523068
Name: data1, dtype: float64

You may use your own aggregation functions.

Pass any function that aggregates an array to the aggregate or 'agg' method

In [ ]:
# peak_to_peak function defined
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)  # using defined function and 'agg' method

,data1,data2
key1,,
a,2.170488,1.300498
b,0.036292,0.487276


In [ ]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.746672  1.109736 -0.204708  0.137118  0.478943  1.222362   
b      2.0 -0.537585  0.025662 -0.555730 -0.546657 -0.537585 -0.528512   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.965781   3.0  0.910916  0.712217  0.092908  0.669671  1.246435   
b    -0.519439   2.0  0.525384  0.344556  0.281746  0.403565  0.525384   

                          
           75%       max  
key1                      
a     1.319920  1.393406  
b     0.647203  0.769023

### Column-Wise and Multiple Function Application


Let's use a tipping dataset to illustrate ideas within this section. 

After loading the dataset with read_csv, add a tipping percentage column called "tip_pct"

#### ** Ensure you have the tips dataset before proceeding! **

In [ ]:
# tips = pd.read_csv('<your path>/tips.csv') # uncomment and replace "your path" with the correct path depending on the dataset location
# Example path shown here:
tips = pd.read_csv('tips.csv')

# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6] # viewing the first six (6) rows of the data

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


You may want to aggregate using a different function depending on the column, or multiple functions at once

We'll see this through a number of examples

In [ ]:
# Group the tips by day and smoker
grouped = tips.groupby(['day', 'smoker'])

In [ ]:
# For descriptive statistics you can pass the name of the function as a string
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean') # mean function passed as a string

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [ ]:
# If you pass a list of functions or function names instead, you get back a DataFrame with column names taken from the functions:

# Passing a number of aggregation functions to 'agg' to evaluate independently on the data groups
grouped_pct.agg(['mean', 'std', peak_to_peak])  # Notice the column headings

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [ ]:
# If you pass a list of (name, function) tuples, the first element of each tuple will be used as the DataFrame column names
# (a list of 2-tuples as an ordered mapping)

grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

With a DataFrame you have more options, as you can specify a list of functions to apply to all of the columns or different functions per column 

In [ ]:
# Running the following three functions on the tip_pct and total_bill for groupings of day and smoker
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

C:\Users\nb3f\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [ ]:
result['tip_pct']   # to view just the portion related to tip_pct

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [ ]:
# Suppose you want to apply different functions to one or more of the columns

# To do this you can pass a dict to 'agg' that contains a mapping of column names and 
# any of the function specifications seen so far

grouped.agg({'tip' : np.max, 'size' : 'sum'})
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In the examples so far, the aggregated data comes back with an index, potentially hierarchical, composed from the unique group key combinations. 

Since this isn't always desirable, you can disable this behavior in most cases by passing "as_index=False" to groupby

In [ ]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

# Notice the hierarchical nature of the row indicies are now gone and you have single rows

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [ ]:
# Look at the output without using as_index for contrast
# Notice the hierarchical nature of the row headers (day and smoker)
tips.groupby(['day', 'smoker']).mean()

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

## Apply: General split-apply-combine

#### The most general-purpose GroupBy method is apply, which is the subject of this section

Apply splits the object being manipulated into pieces, invokes the passed function on each piece, and then attempts to concatenate the pieces together

(Using the tipping dataset from before)

In [ ]:
# Suppose you wanted to select the top five tip_pct values by group

# First write a function that selects the rows with the largest values in a particular column
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [ ]:
# Now if we group by smoker and call apply with this function, 
# we get the the largest values within the two categories of smoker "No" and "Yes"

tips.groupby('smoker').apply(top)

# How did this come together?  The top function is called on each row group from the DataFrame
# and then the results are glued together using pandas.concat, labeling the pieces with the group names
# The result, of couse, has a hierarchical index

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [ ]:
# If you pass a function to apply that takes other arguments or keywords, you can pass these after the function
# The following example illustrates this

tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill') # additional arguments: n=1 this time & column='total_bill'

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

You can call 'describe' function on a GroupBy object as the following example illustrates

In [ ]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [ ]:
400000000/11000

36363.63636363636

In [ ]:
result.unstack('smoker')  # Another way of looking at it

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

Inside GroupBy, when you invoke a method like describe, it is just a short-cut for the following

f = lambda x: x.describe()

grouped.apply(f)

#### The rest of the chapter has a number of examples that are worth reviewing on your own.
